# Download YouTube Dataset
**Tự động download và giải nén**

In [28]:
!pip install kagglehub -q


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
import kagglehub
import os

print("📥 Downloading...")
csv_path = kagglehub.dataset_download(
    "rsrishav/youtube-trending-video-dataset",
    path="US_youtube_trending_data.csv"
)
json_path = kagglehub.dataset_download(
    "rsrishav/youtube-trending-video-dataset",
    path="US_category_id.json"
)

📥 Downloading...


In [30]:
import zipfile
import gzip
import shutil

os.makedirs('./data', exist_ok=True)

print("Processing...")

# Detect file type và giải nén
def extract_file(input_path, output_path):
    with open(input_path, 'rb') as f:
        magic = f.read(2)
    
    try:
        # ZIP file (PK)
        if magic == b'PK':
            print("  → Detected: ZIP")
            with zipfile.ZipFile(input_path, 'r') as z:
                names = z.namelist()
                csv_name = [n for n in names if n.endswith('.csv')][0]
                with z.open(csv_name) as zf:
                    with open(output_path, 'wb') as out:
                        shutil.copyfileobj(zf, out)
        # GZIP file
        elif magic == b'\x1f\x8b':
            print("  → Detected: GZIP")
            with gzip.open(input_path, 'rb') as gz:
                with open(output_path, 'wb') as out:
                    shutil.copyfileobj(gz, out)
        # Plain CSV
        else:
            print("  → Detected: Plain file")
            shutil.copy(input_path, output_path)
    except Exception as e:
        print(f"  ⚠️ Error: {e}")
        shutil.copy(input_path, output_path)

extract_file(csv_path, './data/raw_data.csv')
shutil.copy(json_path, './data/category_id.json')


Processing...
  → Detected: ZIP


'./data/category_id.json'

In [31]:
import pandas as pd

df = pd.read_csv('./data/raw_data.csv')
print(f"Dataset: {len(df):,} rows x {len(df.columns)} cols")
print(f"\nColumns:\n{list(df.columns)}\n")
print("First 3 rows:")
df.tail(3)


Dataset: 268,787 rows x 16 cols

Columns:
['video_id', 'title', 'publishedAt', 'channelId', 'channelTitle', 'categoryId', 'trending_date', 'tags', 'view_count', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'description']

First 3 rows:


,video_id,title,publishedAt,channelId,channelTitle,categoryId,trending_date,tags,view_count,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,description
268784,pRAsQss17xs,12 Things NO-ONE Tells You About Japan | Feat....,2024-04-07T14:53:25Z,UCHL9bfHTxCMi-7vfxQ-AYtg,Abroad in Japan,24,2024-04-15T00:00:00Z,japan|cdawgva|abroad in japan|trash taste,873960,41941,0,1918,https://i.ytimg.com/vi/pRAsQss17xs/default.jpg,False,False,We reveal 12 things no-one tells you to expect...
268785,84QeCD7y_Ds,Buying My Dream Car At 21,2024-04-07T18:00:03Z,UChQ-d14tHB62nskossxO0tw,Joe Fazer,24,2024-04-15T00:00:00Z,gym|youtube|bodybuilder|powerlifter|vlog|vlogg...,535842,42845,0,1852,https://i.ytimg.com/vi/84QeCD7y_Ds/default.jpg,False,False,Use code JOE at checkout to get discounts on M...
268786,SBTA3Tegacs,Murder Drones Training Video,2024-04-05T19:00:12Z,UCn_FAXem2-e3HQvmK-mOH4g,GLITCH,1,2024-04-15T00:00:00Z,meta runner|glitch productions|funny|video gam...,3073412,246678,0,14535,https://i.ytimg.com/vi/SBTA3Tegacs/default.jpg,False,False,J IS FINALLY A PLUSH and NEW Corporate Murder ...


In [32]:
# Sort data theo trending_date (mới nhất → cũ nhất)
df['trending_date'] = pd.to_datetime(df['trending_date'])
df_sorted = df.sort_values('trending_date', ascending=False)

print("Date Range:")
print(f"  Newest: {df_sorted['trending_date'].iloc[0]}")
print(f"  Oldest: {df_sorted['trending_date'].iloc[-1]}")
print(f"\nTop 5 most recent videos:")
df_sorted.head(5)[['trending_date', 'title', 'channelTitle', 'view_count']]

Date Range:
  Newest: 2024-04-15 00:00:00+00:00
  Oldest: 2020-08-12 00:00:00+00:00

Top 5 most recent videos:


,trending_date,title,channelTitle,view_count
268786,2024-04-15 00:00:00+00:00,Murder Drones Training Video,GLITCH,3073412
268649,2024-04-15 00:00:00+00:00,I DESTROYED A ROLLS ROYCE TO REBUILD MARCUS RA...,Mat Armstrong MK2,1937871
268659,2024-04-15 00:00:00+00:00,Everything is gone...,Drift Games,1067988
268658,2024-04-15 00:00:00+00:00,I Cheated With SCARY CAMERA In Build Battle...,Quiff,535793
268657,2024-04-15 00:00:00+00:00,"Julian Newman, D’Aydrian, Paul Pierce vs Josh ...",Creator League,343628
